In [ ]:
import os


In [ ]:
NEWS_API_KEY="add apikey"

In [ ]:
import requests

def get_news(topic, api_key, num_articles=3):
    url = "https://newsapi.org/v2/everything"
    params = {
        "q": topic,
        "language": "en",
        "sortBy": "publishedAt",
        "pageSize": num_articles,
        "apiKey": api_key,
    }

    response = requests.get(url, params=params)

    if response.status_code != 200:
        print("Failed to fetch news:", response.status_code, response.text)
        return []

    data = response.json()
    articles = data.get("articles", [])

    result = []
    for article in articles:
        result.append({
            "title": article.get("title", ""),
            "description": article.get("description", ""),
            "url": article.get("url", "")
        })

    return result


In [ ]:
!pip install mistralai

In [ ]:
import mistralai
print(mistralai.__version__)


In [ ]:
from huggingface_hub import InferenceClient

HF_API_KEY = "add apikey"


In [ ]:
import re
from huggingface_hub import InferenceClient

def generate_news_script_with_mistral(news_data):
    article_blocks = ""
    for i, article in enumerate(news_data, 1):
        article_blocks += f"{i}. {article['title']}\n{article['description']}\n\n"

    prompt = f"""
You are a warm, professional news anchor delivering the evening bulletin with clarity and confidence.
You are speaking directly to viewers as if live on air.

Given the following news headlines and descriptions, create a cohesive, engaging spoken script that:
- Begins with a warm, friendly greeting and introduces Julius by name.
- Naturally flows from one story to the next — do not number them or mention sections like "in tech news".
- Uses smooth transitions that sound like human speech.
- Ends with a warm, thoughtful sign-off.
- Avoids any formatting (no bullet points, no headings, no [pause] or stage directions).
- Do not call this a script or refer to your own writing.

Just return the final speech text as Julius would say it aloud on live television.

News Articles:
{article_blocks}
"""

    client = InferenceClient(
        model="mistralai/Mistral-7B-Instruct-v0.2",
        token=HF_API_KEY
    )

    response = client.chat_completion(
        messages=[
            {"role": "system", "content": "You are a helpful news script writing assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=800,
        temperature=0.7
    )

    output = response.choices[0].message["content"].strip()


    output = re.sub(r"^(?:\[.*?\]|.*?script.*?:|julius:)\s*", "", output, flags=re.IGNORECASE)
    output = re.sub(r"^(?:\[.*?\]|.*?script.*?:|Transition:)\s*", "", output, flags=re.IGNORECASE)
    return output.strip()


In [ ]:
!pip install gTTS

In [ ]:
!pip install --upgrade diffusers transformers accelerate safetensors


In [ ]:
!pip install requests pillow


In [ ]:
from gtts import gTTS
from IPython.display import Audio, display
from IPython.display import Audio, display
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
import matplotlib.pyplot as plt

def speak_in_colab(script_text):
    tts = gTTS(text=script_text, lang='en')
    tts.save("temp.mp3")
    print(" Ready to play:")
    display(Audio("temp.mp3", autoplay=True))  # Autoplay audio in Colab cell


In [ ]:

def load_image_pipeline():
    return StableDiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2", torch_dtype=torch.float16, revision="fp16"
    ).to("cuda")


def generate_and_display_images(pipe, articles, count=5):
    selected = articles[:count]
    for article in selected:
        prompt = f"illustrative graphics, charts, or pictures for: {article['title']} - {article['description']}"
        print(f" Generating image for: {article['title']}")
        img = pipe(prompt).images[0]


        plt.figure(figsize=(5, 5))
        plt.imshow(img)
        plt.axis("off")
        plt.title(article['title'], fontsize=12)
        plt.show()


In [ ]:

def main():
    topic = "hollywood"
    articles = get_news(topic, NEWS_API_KEY, num_articles=10)

    if not articles:
        print("No news articles fetched.")
        return

    script = generate_news_script_with_mistral(articles)
    print("\n Final Script by Julius:\n", script)


    speak_in_colab(script)

    pipe = load_image_pipeline()
    generate_and_display_images(pipe, articles, count=10)


if __name__ == "__main__":
    main()